In [1]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
import random

import csv
import shutil

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from tqdm.notebook import tqdm, trange
from scipy.spatial.transform import Rotation as rotation

plt.rcParams['figure.figsize'] = (20, 10)
#https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

In [2]:
def to_cv_image(image):
    image = (image.transpose(1, 2, 0) * 255).astype(np.uint8)
    return image

def to_cv_kp(kp):
    return list(map(lambda x: cv.KeyPoint(x[1], x[0], 0), kp))

def draw_cv_kp(image, kp, color=(0, 255, 0)):
    return cv.drawKeypoints(image, kp, None, color=color)


def to_cv_dmatch(match_indices):
    return list(map(lambda x: cv.DMatch(x[0], x[1], 0, 0), zip(np.arange(0, len(match_indices)), match_indices)))


def draw_cv_matches(image1, image2,
                    kp1, kp2,
                    match_indices,
                    match_mask=None, match_color=(0, 255, 0), single_point_color=(255, 0, 0)):
    cv_match_indices = to_cv_dmatch(match_indices)
    
    if match_mask is not None:
        flags = cv.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS
        
        match_mask = match_mask.tolist()
        
    else:
        flags = None
    
    return cv.drawMatches(image1, kp1, image2, kp2, cv_match_indices, None, 
                          matchColor=match_color, 
                          singlePointColor=single_point_color,
                          matchesMask=match_mask,
                         flags=flags)


def plot_figures(figures, nrows=1, ncols=1, size=(18, 18)):
    """Plot a dictionary of figures.

    Parameters
    ----------
    figures : <title, figure> dictionary
    ncols : number of columns of subplots wanted in the display
    nrows : number of rows of subplots wanted in the figure
    """

    fig, axes_list = plt.subplots(ncols=ncols, nrows=nrows, figsize=size)
    for ind, title in zip(range(len(figures)), figures):
        if nrows * ncols != 1:
            axes_list.ravel()[ind].imshow(figures[title], cmap='gray')
            axes_list.ravel()[ind].set_title(title)
            axes_list.ravel()[ind].set_axis_off()
        else:
            axes_list.imshow(figures[title], cmap='gray')
            axes_list.set_title(title)
            axes_list.set_axis_off()

    plt.tight_layout()
    return fig
    


In [3]:
def rgb2gray(img):
    return cv.cvtColor(img, cv.COLOR_BGR2GRAY)

def bgr2rgb(img):
    return cv.cvtColor(img, cv.COLOR_BGR2RGB)

def GaussianFilter(img, gausFil):
    return cv.GaussianBlur(imgGray, gausFil, 1)

def darw(img1, img2, title = "Results"):
    cv.imshow(title, np.hstack([img1, img2]))


In [30]:
def keyPoints(img, type):
    tmp = img.copy()
    imgGray = rgb2gray(bgr2rgb(tmp))

    if type == "SIFT":
        sift = cv.SIFT_create(nfeatures=8000)
        kp = sift.detect(imgGray,None)
        kp, des = sift.detectAndCompute(imgGray,None)
        imgRes = cv.drawKeypoints(imgGray,kp,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    elif type == "ORB":
        orb = cv.ORB_create()
        kp, des = orb.detectAndCompute(imgGray,None)
        imgRes = cv.drawKeypoints(imgGray,kp,tmp,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    return np.array(kp), np.array(des), imgRes

# nnMatching

In [31]:
def nnMatching(image1, image2,kp1, kp2, kp1_desc, kp2_desc, num = -1):
    #m = min(len(kp1_desc),len(kp2_desc))
    #R = np.arange(m)
    #dist = np.linalg.norm(kp1_desc[R[0:],None,:] - kp2_desc[None,R[0:],:], axis = -1)
    
    dist = np.linalg.norm(kp1_desc[:,None,:] - kp2_desc[None,:,:], axis = -1)

    nn_idx1 = dist.argmin(axis = 1)

    Res = plot_figures({'nn matching': draw_cv_matches(image1, image2, 
                                                      kp1, kp2, 
                                                      nn_idx1[0:num])})
    return Res

# Low Ratio

In [32]:
def lowRatio(image1, image2, kp1, kp1_desc, kp2, kp2_desc, r = 0.95, num = -1):
    #m = min(len(kp1_desc),len(kp2_desc))
    #R = np.arange(m)
    #dist = np.linalg.norm(kp1_desc[R,None,:] - kp2_desc[None,R,:], axis = -1)

    dist = np.linalg.norm(kp1_desc[:,None,:] - kp2_desc[None,:,:], axis = -1)
    
    nn_idx1 = dist.argmin(axis = -1)
    
    nn_dist1 = np.sort(dist, axis=-1)
    ratio1 = nn_dist1[:,0]/nn_dist1[:,1]
    ratio_test_mask1 = ratio1 < r 

    Res = plot_figures({'Low Rartio': draw_cv_matches(image1, image2,
                                                                       kp1, kp2,
                                                                       nn_idx1[0:num],
                                                                       ratio_test_mask1.astype(int)[0:num])})
    return Res


def mutual_lowRatio(image1, image2, kp1, kp1_desc, kp2, kp2_desc, r = 0.95, num = -1):
    #m = min(len(kp1_desc),len(kp2_desc))
    #R = np.arange(m)
    #dist = np.linalg.norm(kp1_desc[R,None,:] - kp2_desc[None,R,:], axis = -1)

    dist = np.linalg.norm(kp1_desc[:,None,:] - kp2_desc[None,:,:], axis = -1)
#     print(dist.shape)
    nn_idx1 = dist.argmin(axis = -1)
    
    nn_dist1 = np.sort(dist, axis=-1)
    ratio1 = nn_dist1[:,0]/nn_dist1[:,1]
    ratio_test_mask1 = ratio1 < r 
    
    nn_idx2 = np.argmin(dist,axis=0)
    nn_dist2=np.sort(dist,axis=0)
    ratio_test_mask2 = (nn_dist2[0,:]/nn_dist2[1,:]<r)
    
    numbers=np.arange(kp1_desc.shape[0])
    mutual_mask = (nn_idx2[nn_idx1[:]]==numbers[:])

    ratio_test_combine=ratio_test_mask1*ratio_test_mask2[nn_idx1]
    match_mask = ratio_test_combine*mutual_mask
    
    kp_ind_filtered=np.zeros((1,2))
    for i in range(len(kp1)):
        if match_mask[i]:
            kp_ind_filtered=np.vstack((kp_ind_filtered,np.array([i,nn_idx1[i]])[None]))
    kp_ind_filtered=kp_ind_filtered[1:].astype(int)
    Res = plot_figures({'Low Rartio': draw_cv_matches(image1, image2,
                                                       kp1, kp2,
                                                       nn_idx1[0:num],
                                                       match_mask.astype(int)[0:num])})
    return Res,kp_ind_filtered

# Brute-Force Matching with ORB Descriptors
https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html

In [33]:
def BFMatching(image1, image2, kp1, kp2, kp1_desc, kp2_desc, num = -1):
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)# create BFMatcher object
    matches = bf.match(kp1_desc,kp2_desc)# Match descriptors.
    matches = sorted(matches, key = lambda x:x.distance)# Sort them in the order of their distance.

    Res = cv.drawMatches(image1,kp1,image2,kp2,matches[:num],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)# Draw first 10 matches.
    Res = plot_figures({'BFMatching(NORM_HAMMING)':Res})
    return Res

# Brute-Force Matching with SIFT Descriptors and Ratio Test

In [34]:
def BFMatSiftLowR(image1, image2, k = 2, r = 0.95,  num = 10):
    kp1, kp1_desc, imgRes1 = keyPoints(image1.copy(), type="SIFT")
    kp2, kp2_desc, imgRes2 = keyPoints(image2.copy(), type="SIFT")

    # BFMatcher with default params
    bf = cv.BFMatcher()
    matches = bf.knnMatch(kp1_desc,kp2_desc,k)
    # Apply ratio test
    
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    Res = cv.drawMatchesKnn(image1,kp1[:num],image2,kp2[:num],good[:num],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    Res = plot_figures({'Brute-Force Matching with SIFT Descriptors and Ratio Test':Res})
    return Res

# adalam

In [35]:
#!pip install torch
#python -m pip install http://download.pytorch.org/whl/cu80/torch-0.2.0.post3-cp35-cp35m-manylinux1_x86_64.whl 
#python -m pip install torchvision
#import torch
#print(torch.__path__)

In [36]:
from adalam import AdalamFilter

class adalam_filter():
    def __init__(self):
        self.matcher = AdalamFilter()

    def get_data(self, kps):
        pts = np.array([k.pt for k in kps], dtype=np.float32)
        ors = np.array([k.angle for k in kps], dtype=np.float32)
        scs = np.array([k.size for k in kps], dtype=np.float32)
        
        return pts, ors, scs
    
    def process(self, im1, im2, kp1, kp2, dsc1, dsc2, num=-1):
        pts1, ors1, scs1 = self.get_data(kp1)
        pts2, ors2, scs2 = self.get_data(kp2)
        
        matches = self.matcher.match_and_filter(k1=pts1, k2=pts2,
                                                o1=ors1, o2=ors2,
                                                d1=dsc1, d2=dsc2,
                                                s1=scs1, s2=scs2,
                                                im1shape=im1.shape[:2], im2shape=im1.shape[:2]).cpu().numpy()
        
        match_indices = np.zeros(len(matches), dtype=int)
        for i, j in enumerate(matches):
            match_indices[i] = j[1]

        
        fig = plot_figures({'adalam matches': draw_cv_matches(im1, im2, 
                                                              np.array(kp1)[matches[:, 0]], kp2,
                                                              match_indices)}) 
        
        return fig,matches


# Results

In [37]:
def readGroundTruth(filename):
    res = {}
    lines = []
    with open(filename, 'r',encoding='utf-8') as f:
        lines = f.readlines()
        
    count = 0
    times = []
    t = []
    q = []
    for line in lines[3:]:
        #print(f'line {count}: {line}')
        timestamp, tx, ty, tz, qx, qy, qz, qw = line.strip().split()
        #times.append(timestamp)
        end_of_name=timestamp.index('.')+3
        timestamp=timestamp[:end_of_name]
        t=np.array([float(tx), float(ty), float(tz)])
        q=np.array([float(qx), float(qy), float(qz), float(qw)])
        #count += 1
        res[timestamp] = [np.array(t), np.array(q)]
    return res

def getImages(path = os.getcwd()):
    images = []
    for p, subdirs, files in os.walk(path):
        for name in files:
            if ".png" in name:
                images.append(name)
    return images

def getDs(fileName):
    images = getImages(fileName)
    DsCorres = pd.read_excel (fileName+"/DS.xls")
    return images, DsCorres

def buildDs(imagesList, numPairs, step):
    N = len(imagesList)
    assert step < N - N/2#(N/2 + step) > N
    
    i =  np.arange(0, int(N/2), 1)
    imagesIndexes = list(zip(i, i+step))
    imagesNames = []
    for first,second in imagesIndexes:
        imagesNames.append([imagesList[first], imagesList[second]])
    
    return imagesNames#[["A","B"],["C","dddd"]]

def getDs1(imagesList, fileName, generate = True, numPairs  = 10, step = 3):    
    global DsCorres, wantedToTake
    if generate:
        selImages = buildDs(imagesList, numPairs, step)
        
        with open(fileName+'/DS.txt', 'w',encoding='utf-8') as f:
            for n1, n2 in selImages:
                f.writelines(n1 + " " + n2+"\n")
        
    DsCorres =[]
    with open(fileName+'/DS.txt', encoding='utf8') as f:
        for line in f:
            name1, name2 = line.strip().split()
            DsCorres.append([name1, name2])

    w = random.sample(range(len(DsCorres)), numPairs)
    wantedToTake = []
    for i in w:
        wantedToTake.append(DsCorres[i])
    #print(w)

    #return ""


In [38]:
def pos_estimation(pts_left, pts_right):
    '''
    pts_left: list of cv2.KeyPoint
    pts_right: list of cv2.KeyPoint
    '''
    fx = 525.0  # focal length x
    fy = 525.0  # focal length y
    cx = 319.5  # optical center x
    cy = 239.5  # optical center y
    intrinsic_mat = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    pts1 = np.array([x.pt for x in pts_left])
    pts2 = np.array([x.pt for x in pts_right])
    #print(pts_left, pts_right, intrinsic_mat)
    M, mask = cv.findEssentialMat(pts1, pts2, intrinsic_mat)
    retval, R, t, mask = cv.recoverPose(M, pts1, pts2, intrinsic_mat)
    
    return t,R

In [39]:
def test(DsCorres, n2show = -1, filename = "/Dataset",directory2save = "/resu"):
    k=0
    p = os.getcwd()+filename
    p2save = os.getcwd() + directory2save
    if os.path.isdir(p2save):
        shutil.rmtree(p2save)
    os.mkdir(p2save)
    
    adalamOBJ=adalam_filter()       
    p_bar = trange(len(DsCorres))
    
#     for name1, name2, x in DsCorres, p_bar:
    lr_tran_sift=np.zeros((3,1))
    lr_rot_sift=np.zeros((1,3,3))
    lr_tran_orb=np.zeros((3,1))
    lr_rot_orb=np.zeros((1,3,3))
    adalam_tran_sift=np.zeros((3,1))
    adalam_rot_sift=np.zeros((1,3,3))
    adalam_tran_orb=np.zeros((3,1))
    adalam_rot_orb=np.zeros((1,3,3))
    
    for i,(names, _x) in enumerate(zip(DsCorres, p_bar)):
        name1, name2 = names
        image1 = bgr2rgb(cv.imread(p+"/"+ name1))
        image2 = bgr2rgb(cv.imread(p+"/"+ name2))
        kp1, kp1_desc, imgRes1 = keyPoints(image1.copy(), type="SIFT")
        kp2, kp2_desc, imgRes2 = keyPoints(image2.copy(), type="SIFT")
        kp3, kp3_desc, imgRes3 = keyPoints(image1.copy(), type="ORB")
        kp4, kp4_desc, imgRes4 = keyPoints(image2.copy(), type="ORB")

#         print("SIFT Descriptors")
#         print("\tShape of kp1_desc: ", kp1_desc.shape)
#         print("\tShape of kp2_desc: ", kp2_desc.shape)
#         print("ORB Descriptors")
#         print("\tShape of kp3_desc: ", kp3_desc.shape)
#         print("\tShape of kp4_desc: ", kp4_desc.shape)

#         orginalImgs = plot_figures({'image1': image1, 'image2': image2}, 1, 2);        

#         ResNnmatching = nnMatching(image1.copy(), image2.copy(), kp1, kp2, kp1_desc, kp2_desc, num = n2show);
#         ResNnmatching.savefig(p2save + "\\" +"Res_Nnmatching " + str(k))
#         plt.close(ResLowRatio)
        
#         ResLowRatio = lowRatio(image1.copy(), image2.copy(), kp1, kp1_desc, kp2, kp2_desc, r  = 1, num = n2show)
#         ResLowRatio.savefig(p2save + "\\" +"Res_LowRatio_SIFT " + str(k))
#         plt.close(ResLowRatio)
        
#         ResLowRatio = lowRatio(image1.copy(), image2.copy(), kp3, kp3_desc, kp4, kp4_desc, r  = 1, num = n2show)
#         ResLowRatio.savefig(p2save + "\\" +"Res_LowRatio_ORB " + str(k))
#         plt.close(ResLowRatio)
 
        ResLowRatio,lr_matches_sift = mutual_lowRatio(image1.copy(), image2.copy(), kp1, kp1_desc, kp2, kp2_desc, r  = 0.95, num = n2show)
        lowratio_time = %timeit -o mutual_lowRatio(image1.copy(), image2.copy(), kp1, kp1_desc, kp2, kp2_desc, r  = 0.95, num = n2show)
        print(f'Iter {i} Lowe ration time consumption: {lowratio_time}')
        ResLowRatio.savefig(p2save + "/" +"Res_mutual_LowRatio_SIFT " + str(k))
        plt.close(ResLowRatio)
        t,r=pos_estimation(kp1[lr_matches_sift[:,0]],kp2[lr_matches_sift[:,1]])
        lr_tran_sift=np.concatenate((lr_tran_sift,t),axis=1)
        lr_rot_sift=np.concatenate((lr_rot_sift,r[None, :, :]),axis=0)
        
        ResLowRatio,lr_matches_orb = mutual_lowRatio(image1.copy(), image2.copy(), kp3, kp3_desc, kp4, kp4_desc, r  = 0.95, num = n2show)
        ResLowRatio.savefig(p2save + "/" +"Res_mutual_LowRatio_ORB " + str(k))
        plt.close(ResLowRatio)
        t,r=pos_estimation(kp1[lr_matches_orb[:,0]],kp2[lr_matches_orb[:,1]])
        lr_tran_orb=np.concatenate((lr_tran_orb,t),axis=1)
        lr_rot_orb=np.concatenate((lr_rot_orb,r[None, :, :]),axis=0)
        
        ResAdalam,adalam_matches_sift=adalamOBJ.process(image1.copy(), image2.copy(), kp1, kp2, kp1_desc, kp2_desc,num = n2show)
        adalam_time = %timeit -o adalamOBJ.process(image1.copy(), image2.copy(), kp1, kp2, kp1_desc, kp2_desc,num = n2show)
        print(f'Iter {i} Lowe ration time consumption: {adalam_time}')
        ResAdalam.savefig(p2save + "/" +"Res_Adalam_SIFT " + str(k))
        plt.close(ResAdalam)
        t,r=pos_estimation(kp1[adalam_matches_sift[:,0]],kp2[adalam_matches_sift[:,1]])
        adalam_tran_sift=np.concatenate((adalam_tran_sift,t),axis=1)
        adalam_rot_sift=np.concatenate((adalam_rot_sift,r[None, :, :]),axis=0)
        
        ResAdalam,adalam_matches_orb=adalamOBJ.process(image1.copy(), image2.copy(), kp3, kp4, kp3_desc, kp4_desc,num = n2show)
        ResAdalam.savefig(p2save + "/" +"Res_Adalam_ORB " + str(k))
        plt.close(ResAdalam)
        t,r=pos_estimation(kp1[adalam_matches_orb[:,0]],kp2[adalam_matches_orb[:,1]])
        adalam_tran_orb=np.concatenate((adalam_tran_orb,t),axis=1)
        adalam_rot_orb=np.concatenate((adalam_rot_orb,r[None, :, :]),axis=0)
        
        
        
#         ResBFMatching = BFMatching(image1.copy(), image2.copy(), kp3, kp4, kp3_desc, kp4_desc, num = n2show);
#         ResBFMatSiftLowR = BFMatSiftLowR(image1.copy(), image2.copy(),k = 2, r = 0.75, num = n2show)
#         plt.close(ResBFMatSiftLowR)
        
        k += 1
    return {'lr_sift': {'tran': lr_tran_sift[:,1:],
                        'rot': lr_rot_sift[1:]},
            'lr_orb': {'tran': lr_tran_orb[:,1:],
                        'rot': lr_rot_orb[1:]},
            'adalam_sift': {'tran': adalam_tran_sift[:, 1:],
                            'rot': adalam_rot_sift[1:]},
            'adalam_orb': {'tran': adalam_tran_orb[:, 1:],
                            'rot': adalam_rot_orb[1:]}}
        #p_bar.set_description(f'{k} pairs is done')

In [40]:
global DsCorres, numPairs, wantedToTake, tran_rot_estimation

dataSetPath = os.getcwd()+"/Dataset"
imagesList = getImages(dataSetPath)
limit = int(len(imagesList) - len(imagesList)/2)
# print(limit)
step = 7
numPairs = 17
getDs1(imagesList, dataSetPath, True, numPairs, step)
print(len(wantedToTake))
# interact(getDs1, imagesList = fixed(imagesList), fileName = fixed(dataSetPath), generate = fixed(True),
#          numPairs = widgets.IntSlider(min=1, max = int(len(imagesList)/2), step=1, value = int(len(imagesList)/2)),
#                      step = widgets.IntSlider(min=1, max=limit, step=1, value=limit));

# testBtn =widgets.Button(
#     value=False,
#     description='Start Testing',
#     disabled=False,
#     button_style='success', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Run experiments',
#     icon='check' # (FontAwesome names without the `fa-` prefix)
# )

# out = widgets.Output()
# # tran_rot_estimation = None
# def on_button_clicked(_):
#     with out:
#         #clear_output()
#         #test(DsCorres,n2show = 20, filename = "\Dataset", directory2save = "\\ggg")
#         tran_rot_estimation = test(wantedToTake,n2show = 20, filename = "/Dataset", directory2save = "/ggg")
#     return tran_rot_estimation
# testBtn.on_click(on_button_clicked)
# widgets.VBox([testBtn,out])
tran_rot_estimation = test(wantedToTake,n2show = 20, filename = "/Dataset", directory2save = "/ggg")


17


  0%|          | 0/17 [00:00<?, ?it/s]

In [41]:
GroundTruth= readGroundTruth(os.getcwd()+"/Dataset/groundtruth.txt")

In [42]:
def get_data_set_pairs(DsCorres):
    data_set_pairs=np.array(DsCorres)
    for pair in data_set_pairs:
        end_of_name=pair[0].index('.')+3
        pair[0]=pair[0][:end_of_name]
        end_of_name=pair[1].index('.')+3
        pair[1]=pair[1][:end_of_name]
    return data_set_pairs

In [43]:
# Compare the data_set_pairs and the groundTruth
from scipy.spatial.transform import Rotation as rotation

def pos_rot_error_GT(data_set_pairs,GroundTruth):
    rot_first2second_all=np.zeros((1,3,3))
    pos_first2second_all=np.zeros((1,3))
    rot_ground2first_all=np.zeros((1,3,3))
    pos_ground2first_all=np.zeros((1,3))
    
    for i in range(len(data_set_pairs)):
        pos_quat_first=GroundTruth[data_set_pairs[i][0]]
        pos_quat_second=GroundTruth[data_set_pairs[i][1]]
        pos_first=pos_quat_first[0]
        pos_second=pos_quat_second[0]
        quat_first=pos_quat_first[1]
        quat_second=pos_quat_second[1]
        
        pos_first2second=pos_second-pos_first
#         print(pos_first2second/np.linalg.norm(pos_first2second))
#         pos_error=np.linalg.norm(pos_first2second)
        quat_first= rotation.from_quat(quat_first)
        quat_second= rotation.from_quat(quat_second)
        rot_first= quat_first.as_matrix()
        rot_second=quat_second.as_matrix()
        rot_first2second=(rot_second)@(rot_first.T)
#         rot_first2second=(rot_first.T)@(rot_second)
#         theta_error=rotation.as_rotvec(rotation_error)
        pos_first2second_all=np.vstack((pos_first2second_all,pos_first2second[None]))
        rot_first2second_all=np.vstack((rot_first2second_all,rot_first2second[None]))
        rot_ground2first_all=np.vstack((rot_ground2first_all,rot_first[None]))
        pos_ground2first_all=np.vstack((pos_ground2first_all,pos_first[None]))
        
    return pos_first2second_all[1:],rot_first2second_all[1:],rot_ground2first_all[1:],pos_ground2first_all[1:]

data_set_pairs=get_data_set_pairs(DsCorres)
pos_diff_GT,rot_diff_GT,rot_ground2first,pos_ground2first=pos_rot_error_GT(data_set_pairs,GroundTruth)
print(pos_diff_GT.shape)
print(rot_diff_GT.shape)


(17, 3)
(17, 3, 3)


In [18]:
rot_diff_GT[0]

array([[ 0.99958006, -0.0010793 ,  0.02895747],
       [ 0.00175902,  0.99972327, -0.02345811],
       [-0.02892414,  0.0234992 ,  0.99930535]])

In [19]:
tran_rot_estimation['lr_sift']['rot'][0]

array([[ 0.99984002, -0.01135548, -0.01381955],
       [ 0.01153516,  0.99984905,  0.01299265],
       [ 0.01366992, -0.01314999,  0.99982009]])

In [20]:
tran_rot_estimation['adalam_sift']['rot'][0]

array([[ 0.9993432 , -0.01743586, -0.03176717],
       [ 0.01773628,  0.99980037,  0.00919989],
       [ 0.03160042, -0.00975728,  0.99945295]])

In [21]:
tran_rot_estimation['adalam_sift']['tran'][:,0]# we don't need it, useless

array([0.03808487, 0.27167512, 0.96163516])

In [44]:
# Calculate the error in pos and rotion between ground truth and real estimation
def pos_rot_error(pos_diff_GT,rot_diff_GT,pos_diff_estimation,rot_diff_estimation,rot_ground2first,pos_ground2first):
    theta_error_all=np.zeros(1)
    pos_error_all=np.zeros(1)
    for pos_GT,pos_estimation,rot_GT,rot_estimation,rot_g2f,pos_g2f in zip(pos_diff_GT,pos_diff_estimation,rot_diff_GT,rot_diff_estimation,rot_ground2first,pos_ground2first):
#         print(pos_estimation.shape)
#         pos_error=np.linalg.norm(pos_GT-pos_estimation)

        rot_modify=np.concatenate((rot_g2f,[[0,0,0]]),axis=0)
        tran_modify=np.concatenate((pos_g2f,np.array([1])),axis=0).reshape((4,1))
        transform_mat=np.concatenate((rot_modify,tran_modify),axis=1)
        pos_estimation_modify=np.concatenate((pos_estimation,np.array([1])),axis=0)
        pos_estimation_ground=np.linalg.inv(transform_mat)@pos_estimation_modify
        pos_estimation_ground=pos_estimation_ground[:-1]
#         print(pos_estimation_ground)
        pos_error=np.arccos(np.dot(pos_GT,pos_estimation_ground)/(np.linalg.norm(pos_GT)*np.linalg.norm(pos_estimation_ground)))
#         print(pos_error)
        rot_error=rot_GT@(rot_estimation.T)
        rot_error=rotation.from_matrix(rot_error)
        theta_error=np.linalg.norm(rot_error.as_rotvec())
        theta_error_all=np.concatenate((theta_error_all,np.array([theta_error])))
        pos_error_all=np.concatenate((pos_error_all,np.array([pos_error])))
    
    return pos_error_all[1:],theta_error_all[1:]

# Rotation and translation estimation

# Kinect intrinsics
errors = dict()
for k in tran_rot_estimation.keys():
    print(f'Computing scores for method {k}')
    estim = tran_rot_estimation[k]
    errors[k] = dict()
#     print(estim['tran'].shape)
    pos_error_all,rot_error_all=pos_rot_error(pos_diff_GT,rot_diff_GT,estim['tran'].T, estim['rot'],rot_ground2first,pos_ground2first)
    errors[k]['pos_error'] = pos_error_all
    errors[k]['rot_error'] = rot_error_all
    
# pos_diff_estimation,rot_diff_estimation=pos_estimation(pts_left,pts_right)
# pos_error_all,rot_error_all=pos_rot_error(pos_diff_GT,rot_diff_GT,pos_diff_estimation,rot_diff_estimation)

Computing scores for method lr_sift
Computing scores for method lr_orb
Computing scores for method adalam_sift
Computing scores for method adalam_orb


In [45]:
np.rad2deg(errors['adalam_sift']['rot_error'])

array([1.89472786, 2.70688751, 1.98119283, 3.38187513, 3.37999497,
       3.15071852, 2.09784958, 3.05978461, 3.78373089, 5.53596457,
       4.42266603, 4.14968708, 6.48630862, 4.88149154, 2.25338555,
       5.55839633, 4.23220279])

In [46]:
np.rad2deg(errors['lr_sift']['rot_error'])

array([1.89557965, 3.28825455, 1.9453104 , 3.46456174, 2.20295288,
       3.3447008 , 1.42594655, 3.19738647, 3.99711002, 5.26970237,
       4.77739348, 4.30363107, 5.90667044, 4.07403124, 2.1938658 ,
       5.62379163, 3.59132427])

In [47]:
np.rad2deg(errors['adalam_sift']['pos_error'])

array([ 56.4145306 , 105.98151186, 120.48043351, 120.18353038,
       123.45660817, 110.22457922,  55.92130757,  60.44650252,
        58.01836185,  65.2840719 ,  64.29506356,  56.10399598,
       118.44270298,  62.99052897, 105.58661902, 115.66543874,
        67.63166784])

In [48]:
np.rad2deg(errors['lr_sift']['pos_error'])

array([ 56.31289116, 126.01007824, 121.3848524 , 119.89904528,
       124.16913019, 110.10458208,  56.48053956,  60.17062106,
        58.75039922,  64.96457813,  63.43299123,  55.03985649,
       119.25079212,  61.0481572 , 111.22849577, 115.73483771,
        59.16233839])